In [71]:
import pickle
import requests
from pandas import DataFrame as DF, Series
import sys
import numpy as np

In [62]:
client_token = '230875850816746|hh1JJh8ebiRPuFb_V85xqto4lNo'
def search(q):
    url = 'https://graph.facebook.com/v2.12/search?'
    params = {'access_token': client_token,
              'q': q,
              'type': 'place',
              'fields': 'category_list,about,website,name,location'}
    r = requests.get(url, params=params).json()['data']
    locations = []
    results = []
    for d in r:
        for k in list(d):
            if k == 'category_list':
                d[k] = ','.join([_['name'] for _ in d[k]])
            elif k == 'location':
                _ = d[k].copy()
                _['id'] = d['id']
                locations.append(_)
                del d[k], _
        results.append(d)
    return DF(results).merge(DF(locations), on='id')

In [4]:
with open("transaction_detail_first_batch_fm.pickle", 'rb') as f:
    fm_first_batch = pickle.load(f)

In [44]:
starting_companies = list(fm_first_batch.keys())

In [54]:
starting_companies = list(fm_first_batch.keys())
matched_companies = []
for company, matches in fm_first_batch.items():
    match_thresh = []
    match_thresh.append(company)
    for match in matches:
        if match[1] > 95:
            match_thresh.append(match[0])
    if len(match_thresh) > 1:
        matched_companies.append(match_thresh)

In [57]:
for row in matched_companies:
    for item in row:
        starting_companies = list(filter(lambda x: x != item, starting_companies))

In [59]:
len(starting_companies)

5948

In [56]:
len(matched_companies)

1236

In [74]:
fb_matched = []
not_fb_matched = []
for i, row in enumerate(matched_companies):
    new_row = []
    for business in row:
        matched = False
        try:
            search(business)
            fb_matched.append(business)
            matched = True
        except KeyError:
            new_row.append(business)
    if not matched:
        not_fb_matched.append(new_row)
    if i % 10 == 0:
        print(i, len(matched_companies), len(not_fb_matched))

0 1236 0
10 1236 0
20 1236 0
30 1236 5
40 1236 9
50 1236 9
60 1236 12
70 1236 16
80 1236 19
90 1236 19
100 1236 22
110 1236 24
120 1236 26
130 1236 29
140 1236 31
150 1236 33
160 1236 35
170 1236 39
180 1236 42
190 1236 48
200 1236 50
210 1236 52
220 1236 58
230 1236 65
240 1236 68
250 1236 70
260 1236 73
270 1236 74
280 1236 77
290 1236 78
300 1236 81
310 1236 83
320 1236 87
330 1236 90
340 1236 94
350 1236 98
360 1236 99
370 1236 102
380 1236 104
390 1236 108
400 1236 112
410 1236 116
420 1236 119
430 1236 122
440 1236 122
450 1236 122
460 1236 124
470 1236 127
480 1236 130
490 1236 133
500 1236 136
510 1236 137
520 1236 141
530 1236 144
540 1236 146
550 1236 148
560 1236 150
570 1236 151
580 1236 152
590 1236 154
600 1236 157
610 1236 158
620 1236 161
630 1236 163
640 1236 165
650 1236 169
660 1236 171
670 1236 173
680 1236 173
690 1236 175
700 1236 177
710 1236 179
720 1236 181
730 1236 184
740 1236 188
750 1236 191
760 1236 194
770 1236 195
780 1236 196
790 1236 199
800 1236 201
8

In [78]:
with open("companties_to_check.csv", 'a') as f:
    for row in not_fb_matched:
        f.write(str.join(", ", row) + "\n")

['Consolidated Holdings (Guild Building', 'Consolidated Holdings Guild Building']
['Gaylored Eyerman Bradley PC']
['S Dipaola Photography', 'S Dipaola Photography']
['Tri-County Investments, LLC', 'Tri County Investments LLC', 'Tri County Investments LLC']
['Hewlett-Packard Finance Operations', 'Hewlett Packard Finance Operations']
['87KW Development, LLC', '87KW Development LLC']
['Newland Real Estate Group, LLC', 'Newland Real Estate Group LLC']
['John Powell & Associates Inc.', 'John Powell & Associates Inc', 'John Powell & Associates Inc']
['PKS INTERNATIONAL LLC', 'PKS international LLC']
['The Greenbrier Cos. Lake Oswego', 'Te Greenbrier Co Lake Oswego']
['Governemt Relations Strategies, Inc.', 'Government relations strategies inc', 'Governmant Relations Strategies Inc', 'Government Relations Strategies Inc', 'Government Relations Strategies Inc']
['Kramer s Metro Mailing Service Inc']
['GOODWIN SIMON VICTORIA RESEARCH', 'Goodwin Simon Victoria Research']
['J.C.Milne Real Propert